In [41]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from sodapy import Socrata
import datetime

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, lit, udf
from secop.pipelines.data_engineering.utilities import _remove_tildes

<IPython.core.display.Javascript object>

In [2]:
%load_ext nb_black
%matplotlib inline

<IPython.core.display.Javascript object>

In [4]:
secop_int = catalog.load("secop_int")

[08/11/22 09:57:33] INFO     Loading data from 'secop_int' (SparkDataSetSECOP)...               ]8;id=687168;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=150964;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

<IPython.core.display.Javascript object>

In [5]:
secop_int.cache()

DataFrame[nivel_entidad: string, nombre_de_la_entidad: string, estado_del_proceso: string, modalidad_de_contrataci_n: string, objeto_a_contratar: string, tipo_de_contrato: string, numero_del_contrato: string, numero_de_proceso: string, valor_contrato: string, nom_raz_social_contratista: string, departamento_entidad: string, municipio_entidad: string, objeto_del_proceso: string, tipo_contrato: string, origen: string, nit_de_la_entidad: string]

<IPython.core.display.Javascript object>

In [6]:
secop_int.columns

[
    'nivel_entidad',
    'nombre_de_la_entidad',
    'estado_del_proceso',
    'modalidad_de_contrataci_n',
    'objeto_a_contratar',
    'tipo_de_contrato',
    'numero_del_contrato',
    'numero_de_proceso',
    'valor_contrato',
    'nom_raz_social_contratista',
    'departamento_entidad',
    'municipio_entidad',
    'objeto_del_proceso',
    'tipo_contrato',
    'origen',
    'nit_de_la_entidad'
]

<IPython.core.display.Javascript object>

In [33]:
secop_int = secop_int.drop(
    "origen", "tipo_contrato", "numero_del_contrato", "numero_de_proceso"
)

<IPython.core.display.Javascript object>

In [7]:
secop_int = secop_int.withColumn("nivel_entidad", F.lower(col("nivel_entidad")))

<IPython.core.display.Javascript object>

In [8]:
secop_int.groupBy("nivel_entidad").count().show()

+-------------+-------+
|nivel_entidad|  count|
+-------------+-------+
|  territorial|1601071|
|  no definido|  32651|
|     nacional|  37586|
+-------------+-------+



<IPython.core.display.Javascript object>

In [9]:
secop_int = secop_int.withColumn(
    "estado_del_proceso", udf(remove_tildes)(F.lower(col("estado_del_proceso")))
)

<IPython.core.display.Javascript object>

In [10]:
count_process_state = secop_int.groupBy("estado_del_proceso").count().toPandas()
count_process_state.sort_values("count", ascending=False)

[08/11/22 10:02:31] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/s ]8;id=368520;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=642091;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py#109\109]8;;\
                             ite-packages/pyspark/sql/pandas/utils.py:35: DeprecationWarning:                      
                             distutils Version classes are deprecated. Use packaging.version                       
                             instead.                                                                              
                               if LooseVersion(pandas.__version__) <                                               
                             LooseVersion(minimum_pandas_version):                                                 
                                                                                                                   

,estado_del_proceso,count
12,celebrado,842075
0,liquidado,468543
16,convocado,128217
5,en ejecucion,68997
13,terminado anormalmente despues de convocado,40781
1,modificado,28693
17,adjudicado,25133
15,terminado sin liquidar,21939
3,activo,11275
9,borrador,10017


<IPython.core.display.Javascript object>

In [11]:
secop_int = secop_int.withColumn(
    "modalidad_de_contrataci_n",
    udf(remove_tildes)(F.lower(col("modalidad_de_contrataci_n"))),
)

<IPython.core.display.Javascript object>

In [12]:
count_process_type = secop_int.groupBy("modalidad_de_contrataci_n").count().toPandas()
count_process_type["prop"] = (
    count_process_type["count"] / count_process_type["count"].sum()
)

[08/11/22 10:02:38] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/s ]8;id=721376;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=766145;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py#109\109]8;;\
                             ite-packages/pyspark/sql/pandas/utils.py:35: DeprecationWarning:                      
                             distutils Version classes are deprecated. Use packaging.version                       
                             instead.                                                                              
                               if LooseVersion(pandas.__version__) <                                               
                             LooseVersion(minimum_pandas_version):                                                 
                                                                                                                   

<IPython.core.display.Javascript object>

https://www.funcionpublica.gov.co/eva/gerentes/Modulo4/tema-2/1-modalidades.html

In [13]:
def clean_modalidad_contratacion(mod: str):
    """Clean and group modalidad de contratacion"""
    if ("concurso de meritos" in mod) or ("concurso_meritos" in mod):
        return "concurso de meritos abiertos"
    elif "regimen especial" in mod:
        return "regimen especial"
    elif ("minima cuantia" in mod) or ("menor cuantia" in mod):
        return "minima cuantia"
    elif "contratacion directa" in mod:
        return "contratacion directa"
    elif "subasta" in mod:
        return "subasta"
    elif ("licitacion publica" in mod) or ("licitacion obra publica" in mod):
        return "licitacion publica"
    else:
        return "Otro"

<IPython.core.display.Javascript object>

In [16]:
count_process_type["modalidad_clean"] = count_process_type[
    "modalidad_de_contrataci_n"
].apply(clean_modalidad_contratacion)
count_process_type

,modalidad_de_contrataci_n,count,prop,modalidad_clean
0,invitacion ofertas cooperativas o asociaciones...,103,0.000062,Otro
1,licitacion publica,26391,0.015791,licitacion publica
2,seleccion abreviada del literal h del numeral ...,292,0.000175,Otro
3,contratacion minima cuantia,375437,0.224637,minima cuantia
4,contratacion directa,120204,0.071922,contratacion directa
5,seleccion abreviada servicios de salud,343,0.000205,Otro
6,contratacion directa (con ofertas),1036,0.000620,contratacion directa
7,minima cuantia,5501,0.003291,minima cuantia
8,contratacion directa menor cuantia,8906,0.005329,minima cuantia
9,subasta,23600,0.014121,subasta


<IPython.core.display.Javascript object>

In [17]:
secop_int = secop_int.withColumn(
    "modalidad_de_contrataci_n",
    udf(clean_modalidad_contratacion)(col("modalidad_de_contrataci_n")),
)

<IPython.core.display.Javascript object>

In [18]:
count_process_type_clean = (
    secop_int.groupBy("modalidad_de_contrataci_n").count().toPandas()
)
count_process_type_clean["prop"] = (
    count_process_type_clean["count"] / count_process_type_clean["count"].sum()
)
count_process_type_clean.sort_values("prop", ascending=False)

[08/11/22 10:06:43] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/s ]8;id=493542;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=306732;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py#109\109]8;;\
                             ite-packages/pyspark/sql/pandas/utils.py:35: DeprecationWarning:                      
                             distutils Version classes are deprecated. Use packaging.version                       
                             instead.                                                                              
                               if LooseVersion(pandas.__version__) <                                               
                             LooseVersion(minimum_pandas_version):                                                 
                                                                                                                   

,modalidad_de_contrataci_n,count,prop
2,contratacion directa,920465,0.550745
4,minima cuantia,454506,0.271946
6,regimen especial,216527,0.129555
1,licitacion publica,31210,0.018674
5,subasta,24520,0.014671
3,concurso de meritos abiertos,19426,0.011623
0,Otro,4654,0.002785


<IPython.core.display.Javascript object>

In [19]:
secop_int = secop_int.withColumn(
    "tipo_de_contrato", udf(remove_tildes)(F.lower(col("tipo_de_contrato")))
)

<IPython.core.display.Javascript object>

In [22]:
count_process_type_cont

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/89/9jx5xq1j4b979h8c76h2dzjr0000gn/T/ipykernel_29098/2337588814.py:1 in <cell line:  │
│ 1>                                                                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/89/9jx5xq1j4b979h8c76h2dzjr0000gn/T/ipykernel_29098/2337588814.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'count_process_type_cont' is not defined

In [20]:
count_process_type_cont = secop_int.groupBy("tipo_de_contrato").count().toPandas()
count_process_type_cont["prop"] = (
    count_process_type_cont["count"] / count_process_type_cont["count"].sum()
)

[08/11/22 10:07:02] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/s ]8;id=776793;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=709259;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py#109\109]8;;\
                             ite-packages/pyspark/sql/pandas/utils.py:35: DeprecationWarning:                      
                             distutils Version classes are deprecated. Use packaging.version                       
                             instead.                                                                              
                               if LooseVersion(pandas.__version__) <                                               
                             LooseVersion(minimum_pandas_version):                                                 
                                                                                                                   

<IPython.core.display.Javascript object>

In [21]:
def clean_tipo_contrato(tip: str):
    """Clean and group tipo de contrato"""
    if ("suministro" in tip) or (tip in ["compraventa", "venta muebles"]):
        return "suministro"
    elif ("arrendamiento" in tip) or ("comodato" in tip):
        return "arrendamiento"
    elif tip in [
        "servicios financieros",
        "credito",
        "fiducia",
        "seguros",
        "emprestito",
    ]:
        return "servicios financieros"
    elif tip in [
        "obra",
        "consultoria",
        "prestacion de servicios",
        "interventoria",
        "concesion",
    ]:
        return tip
    else:
        return "Otro"

<IPython.core.display.Javascript object>

In [22]:
count_process_type_cont["tipo_de_contrato_clean"] = count_process_type_cont[
    "tipo_de_contrato"
].apply(clean_tipo_contrato)

<IPython.core.display.Javascript object>

In [23]:
count_process_type_cont

,tipo_de_contrato,count,prop,tipo_de_contrato_clean
0,suministro,204990,1.226524e-01,suministro
1,servicios financieros,22,1.316334e-05,servicios financieros
2,compraventa,71510,4.278685e-02,suministro
3,venta muebles,45,2.692502e-05,suministro
4,prestacion de servicios,1137451,6.805753e-01,prestacion de servicios
5,concesion,556,3.326736e-04,concesion
6,arrendamiento de inmuebles,894,5.349104e-04,arrendamiento
7,comodato,3010,1.800985e-03,arrendamiento
8,arrendamiento,13824,8.271366e-03,arrendamiento
9,otro,4464,2.670962e-03,Otro


<IPython.core.display.Javascript object>

In [24]:
secop_int = secop_int.withColumn(
    "tipo_de_contrato",
    udf(clean_tipo_contrato)(col("tipo_de_contrato")),
)

<IPython.core.display.Javascript object>

In [25]:
count_process_type_cont_clean = secop_int.groupBy("tipo_de_contrato").count().toPandas()
count_process_type_cont_clean["prop"] = (
    count_process_type_cont_clean["count"]
    / count_process_type_cont_clean["count"].sum()
)
count_process_type_cont_clean.sort_values("prop", ascending=False)

[08/11/22 10:08:40] WARNING  /opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/s ]8;id=191263;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=260831;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/warnings.py#109\109]8;;\
                             ite-packages/pyspark/sql/pandas/utils.py:35: DeprecationWarning:                      
                             distutils Version classes are deprecated. Use packaging.version                       
                             instead.                                                                              
                               if LooseVersion(pandas.__version__) <                                               
                             LooseVersion(minimum_pandas_version):                                                 
                                                                                                                   

,tipo_de_contrato,count,prop
3,prestacion de servicios,1137451,0.680575
0,suministro,278683,0.166745
6,obra,118003,0.070605
1,Otro,76158,0.045568
7,consultoria,23869,0.014282
5,arrendamiento,17780,0.010638
8,interventoria,17636,0.010552
2,servicios financieros,1172,0.000701
4,concesion,556,0.000333


<IPython.core.display.Javascript object>

In [37]:
secop_int = secop_int.withColumn(
    "valor_contrato", col("valor_contrato").cast("integer")
)

<IPython.core.display.Javascript object>

In [39]:
secop_int = secop_int.withColumn(
    "nit_de_la_entidad", udf(lambda x: int(x.split("-")))(col("nit_de_la_entidad"))
)

<IPython.core.display.Javascript object>

In [40]:
secop_int.columns

[
    'nivel_entidad',
    'nombre_de_la_entidad',
    'estado_del_proceso',
    'modalidad_de_contrataci_n',
    'objeto_a_contratar',
    'tipo_de_contrato',
    'valor_contrato',
    'nom_raz_social_contratista',
    'departamento_entidad',
    'municipio_entidad',
    'objeto_del_proceso',
    'nit_de_la_entidad'
]

<IPython.core.display.Javascript object>

In [42]:
for c in [
    "nom_raz_social_contratista",
    "departamento_entidad",
    "municipio_entidad",
    "objeto_a_contratar",
    "objeto_del_proceso",
]:
    secop_int = secop_int.withColumn(c, F.lower(col(c)))
    if c in [
        "nom_raz_social_contratista",
        "departamento_entidad",
        "municipio_entidad",
    ]:
        secop_int = secop_int.withColumn(c, udf(_remove_tildes)(col(c)))

<IPython.core.display.Javascript object>

In [7]:
secop_int_log_in = catalog.load('secop_int_log_in')

[08/11/22 16:33:27] INFO     Loading data from 'secop_int_log_in' (JSONDataSet)...              ]8;id=652094;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=934030;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [8]:
t = 0
p = 0
for k in secop_int_log_in.keys():
    if secop_int_log_in[k]['success'] == 1:
        p+=1
    t+=1

In [9]:
p/t

0.6402172858667184

In [10]:
t

10309

In [11]:
import pandas as pd
from sodapy import Socrata
from typing import Dict
import datetime
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from secop.pipelines.data_engineering.utilities import (
    COLS_SEC_2,
    schema_secop_int,
    _get_nits_to_extract,
    _remove_tildes,
    _clean_modalidad_contratacion,
    _clean_tipo_contrato,
)
from pyspark.sql.types import StructType
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf

In [12]:
secop_int = catalog.load('secop_int')

[08/11/22 16:38:53] INFO     Loading data from 'secop_int' (SparkDataSetSECOP)...               ]8;id=8912;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=440639;file:///opt/homebrew/Caskroom/miniforge/base/envs/dm_project/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [13]:
secop_int.columns

[
    'nivel_entidad',
    'nombre_de_la_entidad',
    'estado_del_proceso',
    'modalidad_de_contrataci_n',
    'objeto_a_contratar',
    'tipo_de_contrato',
    'valor_contrato',
    'nom_raz_social_contratista',
    'departamento_entidad',
    'municipio_entidad',
    'objeto_del_proceso',
    'fecha_de_firma_del_contrato',
    'fecha_inicio_ejecucion',
    'fecha_fin_ejecucion',
    'nit_de_la_entidad'
]

In [ ]:
secop_int.select()

In [ ]:
# To lower case and remove spainsh accent
for c in [
    "nom_raz_social_contratista",
    "departamento_entidad",
    "municipio_entidad",
    "objeto_a_contratar",
    "objeto_del_proceso",
    "nivel_entidad",
    "estado_del_proceso",
    "modalidad_de_contrataci_n",
    "tipo_de_contrato",
    "nombre_de_la_entidad"
]:
    secop_int = secop_int.withColumn(c, F.lower(col(c)))
    if c in [
        "nom_raz_social_contratista",
        "departamento_entidad",
        "municipio_entidad",
        "estado_del_proceso",
        "modalidad_de_contrataci_n",
        "tipo_de_contrato",
        "nombre_de_la_entidad"
    ]:
        secop_int = secop_int.withColumn(c, udf(_remove_tildes)(col(c)))
secop_int = secop_int.withColumn(
    "modalidad_de_contratacion",
    udf(_clean_modalidad_contratacion)(col("modalidad_de_contrataci_n")),
)
secop_int = secop_int.drop("modalidad_de_contrataci_n")
secop_int = secop_int.withColumn(
    "tipo_de_contrato",
    udf(_clean_tipo_contrato)(col("tipo_de_contrato")),
)
secop_int = secop_int.withColumn(
    "valor_contrato", col("valor_contrato").cast("integer")
)
secop_int = secop_int.withColumn(
    "nit_de_la_entidad",
    udf(lambda x: int(x.replace(".", "").split("-")[0]))(col("nit_de_la_entidad")),
)

In [ ]:

def clean_secop_int(secop_int: SparkDataFrame):
    """Clean secop integrated database"""
    # To lower case and remove spainsh accent
    for c in [
        "nom_raz_social_contratista",
        "departamento_entidad",
        "municipio_entidad",
        "objeto_a_contratar",
        "objeto_del_proceso",
        "nivel_entidad",
        "estado_del_proceso",
        "modalidad_de_contrataci_n",
        "tipo_de_contrato",
    ]:
        secop_int = secop_int.withColumn(c, F.lower(col(c)))
        if c in [
            "nom_raz_social_contratista",
            "departamento_entidad",
            "municipio_entidad",
            "estado_del_proceso",
            "modalidad_de_contrataci_n",
            "tipo_de_contrato",
        ]:
            secop_int = secop_int.withColumn(c, udf(_remove_tildes)(col(c)))
    secop_int = secop_int.withColumn(
        "modalidad_de_contratacion",
        udf(_clean_modalidad_contratacion)(col("modalidad_de_contrataci_n")),
    )
    secop_int = secop_int.drop("modalidad_de_contrataci_n")
    secop_int = secop_int.withColumn(
        "tipo_de_contrato",
        udf(_clean_tipo_contrato)(col("tipo_de_contrato")),
    )
    secop_int = secop_int.withColumn(
        "valor_contrato", col("valor_contrato").cast("integer")
    )
    secop_int = secop_int.withColumn(
        "nit_de_la_entidad",
        udf(lambda x: int(x.replace(".", "").split("-")[0]))(col("nit_de_la_entidad")),
    )
    return secop_int


In [19]:
secop_int_pd = pd.read_parquet('/Volumes/TOSHIBA EXT/Secop/data/01_raw/secop_int')

<a href="https://www.funcionpublica.gov.co/eva/gestornormativo/norma.php?i=304">Ley 80</a>